<a href="https://colab.research.google.com/github/aymenchibouti/PrintGit/blob/master/merged%20datakdd2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Read CSV files
log_train = pd.read_csv("log_train spliting.csv")
enrollment_train = pd.read_csv("enrollment_train.csv")
date = pd.read_csv("date.csv")
truth_train = pd.read_csv("truth_train.csv", header=None, names=["enrollment_id", "dropout"])
object_data = pd.read_csv("object.csv")

# Merge dataframes
merged_df = enrollment_train.merge(log_train, on="enrollment_id", how="left")
merged_df = merged_df.merge(date, on="course_id", how="left")
merged_df = merged_df.merge(truth_train, on="enrollment_id", how="left")

# Convert time columns to datetime
merged_df['time'] = pd.to_datetime(merged_df['time'])
merged_df['from'] = pd.to_datetime(merged_df['from'])
merged_df['to'] = pd.to_datetime(merged_df['to'])

# Create features
merged_df['duration'] = (merged_df['to'] - merged_df['from']).dt.total_seconds() / (24 * 60 * 60)
merged_df['days_active'] = (merged_df['time'] - merged_df['from']).dt.total_seconds() / (24 * 60 * 60)

# Group by enrollment_id and calculate aggregate features
grouped = merged_df.groupby('enrollment_id').agg({
    'time': 'count',
    'days_active': 'max',
    'duration': 'first',
    'dropout': 'first'
}).reset_index()

grouped.columns = ['enrollment_id', 'event_count', 'days_active', 'course_duration', 'dropout']

# Calculate correlations
correlation_matrix = grouped.corr()

print("Correlation Matrix:")
print(correlation_matrix)

# Describe the features
print("\nFeature Description:")
print(grouped.describe())

Correlation Matrix:
                 enrollment_id  event_count  days_active  course_duration  \
enrollment_id         1.000000    -0.116363    -0.021934              NaN   
event_count          -0.116363     1.000000     0.280383              NaN   
days_active          -0.021934     0.280383     1.000000              NaN   
course_duration            NaN          NaN          NaN              NaN   
dropout               0.269829    -0.104360    -0.440805              NaN   

                  dropout  
enrollment_id    0.269829  
event_count     -0.104360  
days_active     -0.440805  
course_duration       NaN  
dropout          1.000000  

Feature Description:
       enrollment_id    event_count  days_active  course_duration  \
count  120542.000000  120542.000000  1132.000000         120542.0   
mean   100364.330748       0.375637    25.813604             29.0   
std     58003.738782       5.532181     4.308308              0.0   
min         1.000000       0.000000     0.623912   

In [7]:
import pandas as pd
import numpy as np

# Read CSV files
log_train = pd.read_csv("log_train spliting.csv")
enrollment_train = pd.read_csv("enrollment_train.csv")
date = pd.read_csv("date.csv")
truth_train = pd.read_csv("truth_train.csv", header=None, names=["enrollment_id", "dropout"])
object_data = pd.read_csv("object.csv")

# Merge dataframes
merged_df = enrollment_train.merge(log_train, on="enrollment_id", how="left")
merged_df = merged_df.merge(date, on="course_id", how="left")
merged_df = merged_df.merge(truth_train, on="enrollment_id", how="left")

# Convert time columns to datetime
merged_df['time'] = pd.to_datetime(merged_df['time'])
merged_df['from'] = pd.to_datetime(merged_df['from'])
merged_df['to'] = pd.to_datetime(merged_df['to'])

# Create features
merged_df['duration'] = (merged_df['to'] - merged_df['from']).dt.total_seconds() / (24 * 60 * 60)
merged_df['days_active'] = (merged_df['time'] - merged_df['from']).dt.total_seconds() / (24 * 60 * 60)

# Group by enrollment_id and calculate aggregate features
grouped = merged_df.groupby('enrollment_id').agg({
    'username': 'first',
    'course_id': 'first',
    'time': 'count',
    'source': lambda x: x.value_counts().to_dict(),
    'event': lambda x: x.value_counts().to_dict(),
    'object': lambda x: x.nunique(),
    'from': 'first',
    'to': 'first',
    'days_active': 'max',
    'duration': 'first',
    'dropout': 'first'
}).reset_index()

# Rename columns
grouped.columns = ['enrollment_id', 'username', 'course_id', 'event_count', 'source_counts',
                   'event_counts', 'unique_objects', 'course_start', 'course_end',
                   'days_active', 'course_duration', 'dropout']

# Explode source_counts and event_counts into separate columns
for source in ['server', 'browser']:
    grouped[f'source_{source}_count'] = grouped['source_counts'].apply(lambda x: x.get(source, 0))

for event in ['access', 'problem', 'wiki', 'discussion', 'navigate', 'page_close', 'video']:
    grouped[f'event_{event}_count'] = grouped['event_counts'].apply(lambda x: x.get(event, 0))

# Drop the dictionary columns
grouped = grouped.drop(['source_counts', 'event_counts'], axis=1)

# Display the first few rows and data info
print(grouped.head())
print("\nDataset Info:")
print(grouped.info())

# Save to CSV
grouped.to_csv("merged_features.csv", index=False)
print("\nMerged features saved to 'merged_features.csv'")


   enrollment_id                          username  \
0              1  9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0   
1              3  1qXC7Fjbwp66GPQc6pHLfEuO8WKozxG4   
2              4  FIHlppZyoq8muPbdVxS44gfvceX9zvU7   
3              5  p1Mp7WkVfzUijX0peVQKSHbgd5pXyl4c   
4              6  dpK33RH9yepUAnyoywRwBt1AJzxGlaja   

                          course_id  event_count  unique_objects course_start  \
0  DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila           77              38   2014-06-12   
1  7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx           52              34   2014-06-19   
2  DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila           20              13   2014-06-12   
3  7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx          142              65   2014-06-19   
4  AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd            6               6   2014-06-04   

  course_end  days_active  course_duration  dropout  source_server_count  \
0 2014-07-11    29.392593             29.0        0                   34   
1 2014-07-18    28.342523           

In [3]:
merged_features = pd.read_csv('merged_features.csv')


In [4]:
merged_features


,enrollment_id,username,course_id,event_count,unique_objects,course_start,course_end,days_active,course_duration,dropout,source_server_count,source_browser_count,event_access_count,event_problem_count,event_wiki_count,event_discussion_count,event_navigate_count,event_page_close_count,event_video_count
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,77,38,2014-06-12,2014-07-11,29.392593,29.0,0,34,43,27,19,0,0,10,15,6
1,3,1qXC7Fjbwp66GPQc6pHLfEuO8WKozxG4,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,52,34,2014-06-19,2014-07-18,28.342523,29.0,0,22,30,20,20,0,6,3,3,0
2,4,FIHlppZyoq8muPbdVxS44gfvceX9zvU7,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,20,13,2014-06-12,2014-07-11,20.481100,29.0,0,15,5,12,0,0,0,3,2,3
3,5,p1Mp7WkVfzUijX0peVQKSHbgd5pXyl4c,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,142,65,2014-06-19,2014-07-18,29.964213,29.0,0,52,90,41,44,0,5,14,17,21
4,6,dpK33RH9yepUAnyoywRwBt1AJzxGlaja,AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd,6,6,2014-06-04,2014-07-03,28.057940,29.0,0,4,2,3,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120537,200898,mAAuxtPoEddxQ2jPmf6WwqHOgdvhqrk2,9zpXzW9zCfU8KGBWkhlsGH8B8czISH4J,0,0,2014-07-03,2014-08-01,NaN,29.0,1,0,0,0,0,0,0,0,0,0
120538,200900,y6i20DJpIul6LihHwgZcWxbdAap5GcBP,9zpXzW9zCfU8KGBWkhlsGH8B8czISH4J,0,0,2014-07-03,2014-08-01,NaN,29.0,1,0,0,0,0,0,0,0,0,0
120539,200901,PPEydg4GXh52QanXuUYKbv8ENUSViBbd,9zpXzW9zCfU8KGBWkhlsGH8B8czISH4J,0,0,2014-07-03,2014-08-01,NaN,29.0,1,0,0,0,0,0,0,0,0,0
120540,200904,7k4xZXZirLFjbh80SpWNqJdTubgGglfv,9zpXzW9zCfU8KGBWkhlsGH8B8czISH4J,0,0,2014-07-03,2014-08-01,NaN,29.0,1,0,0,0,0,0,0,0,0,0
